## 为什么会报错？
```
RuntimeError: Output 0 of UnbindBackward0 is a view and its base or another view of its base has been modified inplace. This view is the output of a function that returns multiple views. Such functions do not allow the output views to be modified inplace. You should replace the inplace operation by an out-of-place one.
```

In [1]:
import torch
import numpy as np
import phi.torch.flow as ptf

import tqdm

device = torch.device('cuda')

In [2]:
BOUNDS = ptf.Box(x=1, y=1, z=1)
RESOLUTION = 100

In [3]:
index1 = 0
index2 = index1 + 5
den1_np = np.load(f'origin/origin_{index1:03d}.npz')['density']
den2_np = np.load(f'origin/origin_{index2:03d}.npz')['density']
den1_gpu = torch.tensor(den1_np, dtype=torch.float32, device=device)
den2_gpu = torch.tensor(den2_np, dtype=torch.float32, device=device)

DENSITY1 = ptf.CenteredGrid(
    values=ptf.wrap(den1_gpu, ptf.spatial('x,y,z')),
    extrapolation=ptf.extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
DENSITY2 = ptf.CenteredGrid(
    values=ptf.wrap(den2_gpu, ptf.spatial('x,y,z')),
    extrapolation=ptf.extrapolation.ZERO_GRADIENT,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
VELOCITY0 = ptf.StaggeredGrid(
    values=0,
    extrapolation=ptf.extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)
INFLOW = ptf.CenteredGrid(
    values=ptf.Sphere(center=ptf.tensor([0.5, 0, 0.5], ptf.channel(vector='x,y,z')), radius=0.05),
    extrapolation=ptf.extrapolation.ZERO,
    bounds=BOUNDS,
    resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
)


# @ptf.jit_compile
def step(v, s, p, dt, inflow_rate, inflow, FORCE):
    s = ptf.advect.mac_cormack(s, v, dt) + inflow_rate * ptf.resample(inflow, to=s, soft=True)
    buoyancy = ptf.resample(s * (0, 0.1, 0), to=v) + ptf.resample(FORCE, to=v)
    v = ptf.advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, p = ptf.fluid.make_incompressible(v, (), ptf.Solve('auto', 1e-3, x0=p))
    return v, s, p


DENSITY = DENSITY1
VELOCITY = VELOCITY0

In [4]:
x = torch.linspace(0.2, 0.8, 3, dtype=torch.float32, device=device)
y = torch.linspace(0.2, 0.8, 3, dtype=torch.float32, device=device)
z = torch.linspace(0.2, 0.8, 3, dtype=torch.float32, device=device)
grid_x, grid_y, grid_z = torch.meshgrid(x, y, z, indexing='ij')

LOCATIONS = torch.stack([grid_x, grid_y, grid_z], dim=-1).reshape(-1, 3)  # [27, 3]
DIRECTIONS = torch.randn_like(LOCATIONS, dtype=torch.float32, device=device, requires_grad=True)  # [27, 3]


# OPTIMIZER = torch.optim.RAdam([DIRECTIONS], lr=0.001)


def my_func(xx):
    ret = A * ptf.math.exp(-ptf.length(xx - loca) ** 2 / (2 * sigma ** 2))
    return ptf.vec(x=ret, y=ret, z=ret)


A = 0.2  # 高斯幅值
sigma = 0.2  # 标准差，控制衰减范围
LAMBDA_FIELDS = []
for loca in LOCATIONS:
    LAMBDA = ptf.CenteredGrid(
        values=my_func,
        extrapolation=ptf.extrapolation.ZERO_GRADIENT,
        bounds=BOUNDS,
        resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
    )
    LAMBDA_FIELDS.append(LAMBDA)

DIRECTION_FIELDS = []
for dire in DIRECTIONS:
    DIRECTION = ptf.CenteredGrid(
        values=ptf.wrap(dire, ptf.channel(vector='x,y,z')),
        extrapolation=ptf.extrapolation.ZERO_GRADIENT,
        bounds=BOUNDS,
        resolution=ptf.spatial(x=RESOLUTION, y=RESOLUTION, z=RESOLUTION),
    )
    DIRECTION_FIELDS.append(DIRECTION)

all_iterations = 100
losses = []
for _ in tqdm.trange(all_iterations):
    # OPTIMIZER.zero_grad()

    first = True
    for _1, _2 in zip(LAMBDA_FIELDS, DIRECTION_FIELDS):
        if first:
            FORCE_FIELD = _1 * _2
            first = False
        else:
            FORCE_FIELD = FORCE_FIELD + _1 * _2


    VELOCITY = ptf.resample(FORCE_FIELD, to=VELOCITY) * 0.2
    DENSITY3 = ptf.advect.mac_cormack(DENSITY, VELOCITY, 0.2) + 0.2 * ptf.resample(INFLOW, to=DENSITY, soft=True)

    loss = torch.nn.functional.mse_loss(DENSITY2.values.native('x,y,z'), DENSITY3.values.native('x,y,z'))
    loss.backward()
    # OPTIMIZER.step()

    print(f'loss: {loss.item()}')
    losses.append(loss.item())

    DENSITY = DENSITY3

  1%|          | 1/100 [00:00<01:37,  1.02it/s]

loss: 0.0001653059880482033


  1%|          | 1/100 [00:01<03:15,  1.97s/it]


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.